In [3]:
# import data
import sys
import pickle
import re
import codecs
import string
import shutil
from win32com import client as wc
import docx

In [1]:
import os
dirPath = r"C:\Users\traffic\Desktop\package\data"
dirPath2 = r'C:\Users\traffic\Desktop\0613網路發件'

# result = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]
# print(result)

from os import listdir
from os.path import isfile, isdir, join
files = listdir(dirPath)
files

['2019070819102180',
 '2019070820370042',
 '2019070820420116',
 '2019070908241629',
 '2019070910140460',
 '2019070915412264',
 '2019070915431594',
 '2019070916032398',
 '2019070919091495',
 '2019070920233979']

In [3]:
## change doc into docx
for i in files:
    try:
        word = wc.Dispatch('Word.Application')
        doc = word.Documents.Open(u'C:\\Users\\traffic\\Desktop\\package\\data\\' + i + '\\' + i + '.doc')        # 目標路徑下的檔案
        doc.SaveAs(u'C:\\Users\\traffic\\Desktop\\package\\data\\' + i + '\\' + i + '.docx', 12, False, "", True, "", False, False, False, False)  # 轉化後路徑下的檔案    
        doc.Close()
    except:
        pass
    #word.Quit()

In [9]:
date_list = []
car_list = []
data = []

for k,i in enumerate(files):
    #獲取文件物件
    file=docx.Document("C:\\Users\\traffic\\Desktop\\package\\data\\" + i +'\\' + i +".docx")
    #print("段落數:"+str(len(file.paragraphs)))#段落數為13，每個回車隔離一段

    car_num = []

    #輸出每一段的內容
    #for para in file.paragraphs:
    #   print(para.text)

    #輸出段落編號及段落內容
    for j in range(len(file.paragraphs)):
        #print("第"+str(i)+"段的內容是："+file.paragraphs[i].text)
        if file.paragraphs[j].text.split(':')[0] != '違規車號' and file.paragraphs[j].text.split(':')[0] != '\n違規車號':
            continue
        else:
            car_num.append(j)
    if len(car_num) != 0:
        date_list.append(file.paragraphs[car_num[0]+1].text.split(':')[1].split('日')[0].replace('年','0').replace('月','')[1:])
    else:
        date_list.append('')
    car = ''  
    for i in car_num:
        car += file.paragraphs[i].text.split(':')[1] + str('、')
    car[:-1]
    car_list.append(car[:-1])
    
    i = files[k]
    file=docx.Document("C:\\Users\\traffic\\Desktop\\package\\data\\" + i +'\\' + i +".docx")
    #print(k)
    try:
        table = file.tables[1]
        keys = None
        for i, row in enumerate(table.rows):
            text = (cell.text for cell in row.cells)

            # Establish the mapping based on the first row
            # headers; these will become the keys of our dictionary
            if i == 0:
                keys = tuple(text)
                continue

            # Construct a dictionary for this row, mapping
            # keys to values for this row
            row_data = dict(zip(keys, text)) 
            if not row_data['回復內文']=='第 二 層決行' and not row_data['回復內文']=='交通隊            核稿               決行\n\xa0\n\xa0\n\xa0\xa0':
                data.append(row_data['回復內文'])
    except KeyError:
        try:
            table = file.tables[1].rows[0].cells[1].text
            data.append(table)
        except:
            table = file.tables[0].rows[1].cells[0].text
            data.append(table)

In [12]:
import pandas as pd
from pandas import Series,DataFrame
data_s = Series(data,name = 'data',index = files)
date_s = Series(date_list,name = 'date',index = files)
car_s = Series(car_list,name = 'car',index = files)
df = pd.concat([date_s,car_s,data_s],axis = 1)
df

,date,car,data
2019070819102180,108076,992-NET普通重型機車、 MUU-3801普通重型機車、 MHP-2011普通重型機車...,陳明倫君您好: \n您108年7月8日檢舉交通違規案件，經檢視所提供之舉證資料，辦理情形如下...
2019070820370042,108076,6002-RV自小客車、 ATW-8520自小客車、 3437-DD租賃小客車,陳明倫君您好: \n您108年7月8日檢舉交通違規案件，經檢視所提供之舉證資料，辦理情形如下...
2019070820420116,108076,AKX-0069自小客車、 602-EYC普通重型機車、 RBS-9507租賃小客車,陳明倫君您好:\n您所檢舉車號AKX-0069自小客車及車號602-EYC普通重型機車於10...
2019070908241629,108079,1159-B5自小客車,梁毓仁君您好:\n您所檢舉車號1159-B5自小客車於108年7月9日6時50分，在花蓮縣南...
2019070910140460,108079,AHF-0186自小客車,曾建清君您好:\n您所檢舉車號AHF-0186自小客車於108年7月9日8時59分，在花蓮縣...
2019070915412264,,0516-SN自小客車,劉瓊璋君您好:\n您所檢舉車號0516-SN自小客車，在花蓮市中山路553號丁丁藥局前違規一...
2019070915431594,108078,857-P9營業小客車,張睿紘君您好:\n您所檢舉車號857-P9營業小客車於108年7月8日9時6分，在花蓮市中山...
2019070916032398,,2888-B5自小客車,劉瓊璋君您好:\n您所檢舉車號2888-B5自小客車，在花蓮市中山路612號 199百貨違規...
2019070919091495,108077,MCN-8859普通重型機車,賴立新君您好:\n您所檢舉車號MCN-8859普通重型機車於108年7月7日17時9分，在花...
2019070920233979,108079,ATS-0279自小客車,李俍禛君您好:\n您所檢舉車號ATS-0279自小客車於108年7月9日16時17分，在花蓮...


In [13]:
## adding hyperlinks

import xlsxwriter

workbook = xlsxwriter.Workbook('Hyperlinks.xlsx')
worksheet = workbook.add_worksheet('Hyperlinks')

month = input('Enter a month : ')
day = input('Enter a day : ')

for i,f in enumerate(files,start=1):   
    worksheet.write_url('A'+str(i), '108結畢案件\\108年' + month + '月'+ '\\' + day + '\\'+ f,string = f)
    worksheet.write('B'+str(i),date_s[f])
    worksheet.write('C'+str(i),car_s[f])
    worksheet.write('D'+str(i),data_s[f])
workbook.close()

Enter a month : 7
Enter a day : 0716


In [52]:
i = files[4]
file=docx.Document("C:\\Users\\traffic\\Desktop\\package\\data\\" + i +'\\' + i +".docx")
file.tables[1].rows[0].cells[1].text

'陳明倫君您好:\n您所檢舉車RBT-3983租賃小客車於108年6月15日17時53分，在花蓮市中山路與福建街交岔路口違規一節，經檢視所提供之舉證資料，違規事實明確，將依違反道路交通管理處罰條例第48條第1項第1款（路邊起駛車輛未使用方向燈）規定製單舉發。\n\t感謝來信  敬祝平安'

In [64]:
i = files[4]
file=docx.Document("C:\\Users\\traffic\\Desktop\\package\\data\\" + i +'\\' + i +".docx")
table = file.tables[1]
keys = None
for i, row in enumerate(table.rows):
    text = (cell.text for cell in row.cells)

            # Establish the mapping based on the first row
            # headers; these will become the keys of our dictionary
    if i == 0:
        keys = tuple(text)
        continue

            # Construct a dictionary for this row, mapping
            # keys to values for this row
    row_data = dict(zip(keys, text))        
    row_data

In [65]:
row_data

{}

In [11]:
data.pop(9)

'交通隊            核稿               決行\n\xa0\n\xa0\n\xa0\n\xa0'

In [8]:
data

['陳明倫君您好: \n您108年7月8日檢舉交通違規案件，經檢視所提供之舉證資料，辦理情形如下：\n一、車號992-NET，MUU-3801，MHP-2011，125HKT，HX7-098，MPW-3530普通重型機車於108年7月6日10時7分，在花蓮市明智街與延平街口違規一節，經檢視所提供之舉證資料，違規事實明確，將依違反道路交通管理處罰條例第55條第1項第3款（紅線臨停）規定製單舉發。\n二、車號3833-T7及ALE-0106自小客車於108年7月6日10時7分，在花蓮市明智街69號(禾湘火鍋前)違規一節，經檢視所提供之舉證資料，違規事實明確，將依違反道路交通管理處罰條例第55條第1項第3款（紅線臨停）規定製單舉發。\n\n感謝來信  敬祝平安\n',
 '陳明倫君您好: \n您108年7月8日檢舉交通違規案件，經檢視所提供之舉證資料，辦理情形如下：\n一、車號6002-RV自小客車於108年7月6日12時18分，在花蓮縣新城鄉縣道193線七星潭段違規一節，經檢視所提供之舉證資料，違規事實明確，將依違反道路交通管理處罰條例第55條第1項第3款（紅線臨停）規定製單舉發。\n二、車號ATW-8520自小客車於108年7月6日12時20分，在花蓮縣新城鄉縣道193線七星潭段違規一節，經檢視所提供之舉證資料，違規事實明確，將分別依違反道路交通管理處罰條例第49條第1項第2款（雙黃線迴轉）及同條例第48條第1項第1款（變換車道未使用方向燈）規定製單舉發。\n三、車號3437-DD租賃小客車於108年7月6日12時21分，在花蓮縣新城鄉七星街112號前違規一節，經檢視所提供之舉證資料，違規事實明確，將依違反道路交通管理處罰條例第55條第1項第3款規定製單舉發。\n\t感謝來信  敬祝平安',
 '陳明倫君您好:\n您所檢舉車號AKX-0069自小客車及車號602-EYC普通重型機車於108年7月6日12時31分，在花蓮市復興街與正義街路口違規一節，經檢視所提供之舉證資料，違規事實明確，將依違反道路交通管理處罰條例第55條第1項第3款（紅線臨停）規定製單舉發。\n您所檢舉車號RBS-9507租賃小客車於108年7月6日12時31分，在花蓮市復興街與正義街路口違規一節，因所提供之舉證資料無法明確判定違規事實，不宜製單舉發。\n\t感謝來信  敬祝平安',
 '梁毓仁君您好